The Strategy is to find the mismatch of the bid-ask of the same product in different exchange.
As we can only make a immediately deal when we buy at the ask price and sell at the bid price, we need the bid price in one exchange greater than the ask price in another exchange.

Further research:
1. Use multithreading to get the latest order book cocurrently
2. Trading cost including that the order we make may change the quantity of the bid-ask
3. risk management to capture abnormal move of the order book
4. REST api or Websocket



Updated:
1. The trading fee is about 0.1% for both exchanges, so the total trading cost for one pair including closing position is 0.4%
2. It's hard to get profit if we do not use leverage as the bid ask reversed spread should larger price*0.4%, so I added 10 leverage (the maximum in exchange).
3. Optimized my code with function.
4. Only leverage 10 can have enough reversed bid ask spread to trade

requirements: <br>
huobi-client-sdk <br>
kucoin-python

In [2]:
from huobi.client.market import MarketClient
from huobi.utils import *
from kucoin.client import Market
import time

In [59]:
#times: trading times 
#rates: fee rates
#symbol: the product, must be the same product
#leverage: leverage level

def Pnl(times=10,rates=[0.001,0.001],symbol=['btcusdt','BTC-USDT'],leverage=10):
    client_h = MarketClient()
    symbol_h, symbol_k = symbol
    depth_size = 1

    client_k = Market(url='https://api.kucoin.com')
    account_h=[0,0]
    account_k=[0,0]
    profit=0
    fees=0
    fee_h, fee_k=rates #kucoin: 0.1%, huobi: 0.1%
    market_price=client_h.get_market_detail(symbol_h).close
    spread=(fee_h+fee_k)*2*market_price/leverage
    #trade 10 times(default)
    while times:
        order_h = client_h.get_pricedepth(symbol_h, 'step0', depth_size)
        order_k = client_k.get_part_order(20,symbol_k)
    
        price_h_a, amount_h_a = order_h.asks[0].price, order_h.asks[0].amount
        price_h_b, amount_h_b = order_h.bids[0].price, order_h.bids[0].amount
    
        price_k_a, amount_k_a = float(order_k['bids'][0][0]), float(order_k['bids'][0][1])
        price_k_b, amount_k_b = float(order_k['asks'][0][0]), float(order_k['asks'][0][1])


        if price_h_a <= (price_k_b-spread):
            min_a=min(amount_h_a, amount_k_b)
            print("Huobi buy: ",price_h_a)
            print("Kucoin sell: ",price_k_b)
            print("Amount: ",min_a)
            account_h[0]+=price_h_a*min_a
            account_h[1]+=min_a
            account_k[0]-=price_k_b*min_a
            account_k[1]-=min_a
            fees+=price_h_a*min_a*fee_h+price_k_b*min_a*fee_k
            times-=1
        elif (price_h_b-spread) >= price_k_a:
            min_a=min(amount_k_a, amount_h_b)
            print("Kucoin buy: ",price_k_a)
            print("Huobi sell: ",price_h_b)
            print("Amount: ",min_a)
            account_h[0]-=price_h_b*min_a
            account_h[1]-=min_a
            account_k[0]+=price_k_a*min_a
            account_k[1]+=min_a
            fees+=price_h_b*min_a*fee_h+price_k_a*min_a*fee_k
            times-=1
        time.sleep(0.01)
    #close position when the bid-ask between 2 exchanges return normal
    while True:
        order_h = client_h.get_pricedepth(symbol_h, 'step0', depth_size)
        order_k = client_k.get_part_order(20,symbol_k)
    
        price_h_a, amount_h_a = order_h.asks[0].price, order_h.asks[0].amount
        price_h_b, amount_h_b = order_h.bids[0].price, order_h.bids[0].amount
    
        price_k_a, amount_k_a = float(order_k['bids'][0][0]), float(order_k['bids'][0][1])
        price_k_b, amount_k_b = float(order_k['asks'][0][0]), float(order_k['asks'][0][1])
        if price_k_a>=price_h_b and price_k_b<=price_h_a: #when normal: both bid < ask 
            if account_h[1]>0:
                close_price=price_h_b
                profit+=close_price*account_h[1]-account_h[0]
                fees+=close_price*abs(account_h[1])*fee_h
            else:
                close_price=price_h_a
                profit+=close_price*account_h[1]-account_h[0]
                fees+=close_price*abs(account_h[1])*fee_h
            if account_k[1]>0:
                close_price=price_k_b
                profit+=close_price*account_k[1]-account_k[0]
                fees+=close_price*abs(account_k[1])*fee_k
            else:
                close_price=price_k_a
                profit+=close_price*account_k[1]-account_k[0]
                fees+=close_price*abs(account_k[1])*fee_k
            break
        time.sleep(0.01)
    return [profit*leverage, fees]

In [60]:
rst=Pnl() #BTC USDT

Kucoin buy:  55159.5
Huobi sell:  55186.15
Amount:  0.0345
Huobi buy:  55243.1
Kucoin sell:  55268.6
Amount:  0.006
Huobi buy:  55243.1
Kucoin sell:  55273.1
Amount:  0.01266998
Huobi buy:  55243.1
Kucoin sell:  55273.1
Amount:  0.1569433
Kucoin buy:  55268.1
Huobi sell:  55291.07
Amount:  0.001
Huobi buy:  55315.84
Kucoin sell:  55337.9
Amount:  0.1383
Kucoin buy:  55359.6
Huobi sell:  55385.66
Amount:  0.0005
Kucoin buy:  55361.7
Huobi sell:  55385.31
Amount:  0.08
Kucoin buy:  55375.6
Huobi sell:  55400.04
Amount:  0.006
Kucoin buy:  55354.4
Huobi sell:  55386.31
Amount:  0.024714


In [61]:
print(f"Total profit: {rst[0]}, total trading cost: {rst[1]}, netting: {rst[0]-rst[1]}")

Total profit: 115.66843314401922, total trading cost: 69.44012582329839, netting: 46.22830732072083


In [62]:
rst=Pnl(symbol=['ethusdt','ETH-USDT'])

Huobi buy:  1765.9
Kucoin sell:  1766.64
Amount:  2.4774
Huobi buy:  1765.99
Kucoin sell:  1766.76
Amount:  2.0
Kucoin buy:  1765.46
Huobi sell:  1766.26
Amount:  2.2642
Kucoin buy:  1765.52
Huobi sell:  1766.26
Amount:  0.6167
Kucoin buy:  1765.03
Huobi sell:  1765.79
Amount:  0.0464609
Kucoin buy:  1767.51
Huobi sell:  1768.38
Amount:  1.6946
Huobi buy:  1769.54
Kucoin sell:  1770.68
Amount:  0.1851
Huobi buy:  1768.49
Kucoin sell:  1769.44
Amount:  2.5088
Huobi buy:  1768.49
Kucoin sell:  1769.44
Amount:  2.5088
Huobi buy:  1769.0
Kucoin sell:  1769.87
Amount:  5.6176918


In [63]:
print(f"Total profit: {rst[0]}, total trading cost: {rst[1]}, netting: {rst[0]-rst[1]}")

Total profit: 170.15732150000986, total trading cost: 108.227269154586, netting: 61.93005234542386


In [73]:
rst=Pnl(symbol=['adausdt','ADA-USDT'])

Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242724
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242841
Kucoin sell:  1.243849
Amount:  289.0638
Huobi buy:  1.242841
Kucoin sell:  1.243849
Amount:  289.0638


In [74]:
print(f"Total profit: {rst[0]}, total trading cost: {rst[1]}, netting: {rst[0]-rst[1]}")

Total profit: 23.86510732801071, total trading cost: 14.378086599739198, netting: 9.48702072827151
